In [1]:
from core import Compiler

In [2]:
drizzle_schema = """
import { integer, pgTable, text } from "drizzle-orm/pg-core";

export const gymEquipmentTable = pgTable("gym_equipment", {
    id: integer().primaryKey().generatedAlwaysAsIdentity(),
    name: text(),
    description: text(),
    price: integer(),
    stock: integer(),
});
""".strip()

In [3]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")

In [4]:
result = compiler.compile_drizzle(drizzle_schema)

In [5]:
result

{'exit_code': 0,
 'stdout': "No config path provided, using default 'drizzle.config.ts'\nReading config file '/app/drizzle.config.ts'\nUsing 'pg' driver for database querying\n[⣷] Pulling schema from database...\n\x1b[2K\x1b[1G[✓] Pulling schema from database...\n[✓] Changes applied\n",
 'stderr': None}

In [6]:
print(result["stdout"])

No config path provided, using default 'drizzle.config.ts'
Reading config file '/app/drizzle.config.ts'
Using 'pg' driver for database querying
[⣷] Pulling schema from database...
[✓] Pulling schema from database...
[✓] Changes applied



In [7]:
typespec_schema = """
import "./helpers.js";

extern dec llm_func(target: unknown, history: valueof int32);

//{{typespec_definitions}}
""".strip()

In [8]:
result = compiler.compile_typespec(typespec_schema)

In [9]:
result

{'exit_code': 0,
 'stdout': 'TypeSpec compiler v0.64.0\n\nCompilation completed successfully.\n\n',
 'stderr': None}

In [10]:
typescript_files = {
    "src/logic/index.ts": "this should fail"
}

In [11]:
result = compiler.compile_typescript(typescript_files)

In [12]:
result

{'exit_code': 2,
 'stdout': "src/logic/index.ts(1,6): error TS1005: ';' expected.\n",
 'stderr': None}

In [13]:
typescript_files["src/logic/index.ts"] = '''
import { GenericHandler } from "../common/handler";
import { dummyHandler } from "./dummy_handler";

export const handlers: {[key: string]: GenericHandler<any[], any>} = {
    'dummy': dummyHandler,
};
'''.strip()

In [14]:
result = compiler.compile_typescript(typescript_files)

In [15]:
result

{'exit_code': 0, 'stdout': None, 'stderr': None}

## Test Server

In [16]:
import requests

In [17]:
ADDR = "http://localhost:8080" # assuming runs with HOST=0.0.0.0 PORT=8080

In [18]:
drizzle_result = requests.post(
    f"{ADDR}/compile/drizzle",
    json={"payload": drizzle_schema},
)

In [19]:
drizzle_result.json()

{'exit_code': 0,
 'stdout': "No config path provided, using default 'drizzle.config.ts'\nReading config file '/app/drizzle.config.ts'\nUsing 'pg' driver for database querying\n[⣷] Pulling schema from database...\n\x1b[2K\x1b[1G[✓] Pulling schema from database...\n[✓] Changes applied\n",
 'stderr': None}

In [20]:
typespec_result = requests.post(
    f"{ADDR}/compile/typespec",
    json={"payload": typespec_schema},
)

In [21]:
typespec_result.json()

{'exit_code': 0,
 'stdout': 'TypeSpec compiler v0.64.0\n\nCompilation completed successfully.\n\n',
 'stderr': None}

In [22]:
typescript_result = requests.post(
    f"{ADDR}/compile/typescript",
    json={"payload": typescript_files},
)

In [23]:
typescript_result.json()

{'exit_code': 0, 'stdout': None, 'stderr': None}